In [3]:
import pandas as pd
import os
import geopy.distance
import numpy as np
from pathlib import Path

In [6]:
bus_station = pd.read_csv('tpe_bus_station_info_after_api_call.csv')
mrt_station = pd.read_csv('mrt_station.csv')
bike_station = pd.read_csv('TPE_bike_station.csv')

In [7]:
bike_station['sarea'] = bike_station['sarea'].replace('臺大公館校區','大安區')

KeyError: 'sarea'

In [5]:
bike_station.rename(
    {
        'sno':'bike_station_id',
        'sna':'station_name',
        'act':'disable',
        'sarea':'district',
        'latitude':'lat',
        'longitude':'lng'

    }
    ,axis=1,
    inplace=True
                    )

In [6]:
bike_station['city_code']='TPE'

bike_station['station_name'] = bike_station['station_name'].apply(lambda x: x.split('_')[1])
bike_station = bike_station[['bike_station_id','station_name',
                             'city_code','district','disable','lng','lat']]

In [7]:
mrt_station.rename(
    {
        'StationPosition.PositionLon':'lng',
        'StationPosition.PositionLat':'lat'
    },
    axis=1,
    inplace=True
)

In [8]:
mrt_dist = []
bus_dist = []
for idx, row in bike_station.iterrows():
    point_1 = row[['lat','lng']].to_numpy()
    mrt_dist.append(int(mrt_station[['lat','lng']].apply(lambda row:
                                 geopy.distance.geodesic(point_1,row.to_numpy()).m,
                                 axis=1).min()))
    bus_dist.append(int(bus_station[['lat','lng']].apply(lambda row:
                                 geopy.distance.geodesic(point_1,row.to_numpy()).m,
                                 axis=1).min()))
bike_station['near_to_bus'] = bus_dist
bike_station['near_to_mrt'] = mrt_dist
del mrt_station,bus_station

In [9]:
bike_station.loc[:,'disable'] = ~bike_station['disable'].astype('bool')

In [10]:
# lend_date
# lend_hour
# weekend
# traffic_count

In [11]:
file_list = os.listdir('./history')
file_list = [file for file in file_list if 'bike_usage_history' in file]
extracted_all = None

In [12]:
for path in file_list:
    ubike_hist = pd.read_csv(Path('./history', path), index_col=0)
    ubike_hist.drop_duplicates(inplace=True)
    ubike_hist.columns = ['lend_time', 'lend_station_name', 'return_time',
                  'return_station_name', 'usage_time', 'source_date']
    ubike_hist['lend_date'] = pd.to_datetime(ubike_hist['lend_time']).dt.date
    ubike_hist['lend_hour'] = pd.to_datetime(ubike_hist['lend_time']).dt.hour
    extracted = ubike_hist.groupby(['lend_station_name','lend_date','lend_hour']).size().reset_index(name='traffic_count')
    extracted['weekend'] = pd.to_datetime(extracted['lend_date']).dt.dayofweek.isin([5,6])
    extracted['weekend'] = np.where(extracted['weekend'],'假日','平日')
    if extracted_all is None:
        extracted_all = extracted.copy(deep=True)
    else:
        extracted_all = pd.concat([extracted_all,extracted], ignore_index=True)
        print(extracted_all.shape)
        # break



(1044177, 5)
(1636941, 5)
(1970990, 5)
(2265762, 5)
(2869230, 5)
(3427880, 5)
(3765100, 5)
(4391923, 5)
(4995673, 5)
(5358538, 5)
(5829758, 5)
(6390806, 5)
(6933762, 5)
(7536607, 5)
(8145386, 5)
(8653967, 5)
(9085001, 5)
(9711682, 5)
(9998161, 5)
(10587761, 5)
(11039495, 5)
(11572567, 5)
(11671075, 5)
(11715495, 5)
(11758563, 5)
(11805196, 5)
(12355591, 5)
(12395303, 5)
(12957261, 5)
(12992196, 5)
(13187220, 5)
(13267195, 5)
(13312080, 5)
(13356874, 5)
(13401583, 5)
(13647791, 5)
(14157857, 5)
(14199784, 5)
(14246514, 5)
(14290721, 5)
(14334694, 5)
(14378786, 5)
(14523639, 5)


In [13]:
extracted_all = bike_station.merge(extracted_all,how='inner',left_on='station_name',right_on='lend_station_name')

In [14]:
extracted_all.drop('lend_station_name',axis=1,inplace=True)

In [15]:
extracted_all.drop(['lng', 'lat'],axis=1,inplace=True)

In [16]:
extracted_all.shape

(14285498, 11)

In [17]:
# bike_station_id
# station_name
# city_code
# district
# disable
# near_to_bus
# near_to_mrt
# lend_date
# lend_hour
# weekend
# traffic_count
extracted_all.columns

Index(['bike_station_id', 'station_name', 'city_code', 'district', 'disable',
       'near_to_bus', 'near_to_mrt', 'lend_date', 'lend_hour', 'traffic_count',
       'weekend'],
      dtype='object')

In [21]:
extracted_all.to_csv('pipeline_1_v1.csv',index=False)